# DLS

In [27]:
# Import libraries
import glob
import os
import pandas as pd
import numpy as np

import micasense.dls as dls
import micasense.imageset as imageset

import warnings
warnings.filterwarnings('ignore')

In [28]:
#Input Directory
inDir = r'C:\Users\cdelgado\Downloads'           
os.chdir(inDir)

In [29]:
#Files
rawFiles = glob.glob('JSTAR-2020\**\**\**\**SET')

In [30]:
#Do the job!
DLS_stats = pd.DataFrame()

for rawFile in rawFiles:
    dataset = rawFile.split("\\")[3]
    season = rawFile.split("\\")[1]
    print('Processing dataset:' + dataset)
    
    #Import Capture data into Pandas DataFrame
    imgset = imageset.ImageSet.from_directory(rawFile)
    data, columns = imgset.as_nested_lists()
    dlsData = pd.DataFrame.from_records(data, index='timestamp', columns=columns)
    
    #Separating ground and flight images
    cutoff_altitude = 997
    dlsData = dlsData[dlsData.altitude>cutoff_altitude]
    
    #location
    location = dlsData[['latitude', 'longitude', 'altitude']]
    tuples = [tuple(x) for x in location.values]
    dlsData['location']=tuples
    
    #pose
    dls_pose = dlsData[['dls-yaw', 'dls-pitch', 'dls-roll']]
    tuples = [tuple(x) for x in dls_pose.values]
    dlsData['dls_pose']=tuples
    
    #Computing Solar orientation
    compute_sun_angle = dlsData[['location', 'dls_pose']]
    compute_sun_angle['utc_time'] = dlsData.index
    dls_orientation_vector = np.array([0,0,-1])
    
    dlsData['solar_zenith']=[np.degrees(dls.compute_sun_angle(x[0],
                          (0,0,0),
                          x[2],
                          dls_orientation_vector)[2]) for x in compute_sun_angle.values]
    
    dlsData['solar_azimuth']=[np.degrees(dls.compute_sun_angle(x[0],
                      (0,0,0),
                      x[2],
                      dls_orientation_vector)[4]) for x in compute_sun_angle.values]
    
    #Computing Irradiance
    irr_col = [col for col in dlsData if col.startswith(("irr"))]  
    dlsData['irr-avg'] = dlsData[irr_col].mean(axis = 1, skipna = True) 

    #Results
    columns = ['irr-avg', 'solar_zenith', 'solar_azimuth']
    local_stats = dlsData[columns].mean()
    local_stats['SEASON'] = season
    local_stats['DATASET'] = dataset
    DLS_stats = DLS_stats.append(local_stats, ignore_index=True)

DLS_stats = DLS_stats.dropna() 

#Save the results
DLS_stats.to_csv('DLS.csv', index = True)

Processing dataset:2017B-1
Processing dataset:2017B-2
Processing dataset:2017B-3
Processing dataset:2017B-4
Processing dataset:2018B-1
Processing dataset:2019A-1
Processing dataset:2019A-2
Processing dataset:2019A-3
Processing dataset:2019A-4
Processing dataset:2019B-1
Processing dataset:2019B-2
Processing dataset:2019B-3
